# Feature engineering

## Dependency management

In [1]:
import pandas as pd
import numpy as np
import os

## Minio Bucket setup

In [2]:
MINIO_ENDPOINT = 'http://minio.idoml.precision.uni.lux'
CLEANED_FILENAME = 'lcld/cleaned_data'
FEATURE_FILENAME = 'lcld/feature_data'

In [3]:
bucket_name = 'datasets'
filepath = f'{bucket_name}/{CLEANED_FILENAME}.csv'

## Load pandas dataframe from S3

In [4]:
df = pd.read_csv(
    f's3://{filepath}',
    storage_options={
        'client_kwargs': {'endpoint_url': MINIO_ENDPOINT}
    }
)

In [5]:
df.head()

,id,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,...,fico_range_high,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies
0,68407277,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,...,679.0,7.0,0.0,2765.0,29.7,13.0,w,Individual,1.0,0.0
1,68355089,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,...,719.0,22.0,0.0,21470.0,19.2,38.0,w,Individual,4.0,0.0
2,68341763,20000.0,60 months,10.78,432.66,B,B4,truck driver,10+ years,MORTGAGE,...,699.0,6.0,0.0,7869.0,56.2,18.0,w,Joint App,5.0,0.0
3,68476807,10400.0,60 months,22.45,289.91,F,F1,Contract Specialist,3 years,MORTGAGE,...,699.0,12.0,0.0,21929.0,64.5,35.0,w,Individual,6.0,0.0
4,68426831,11950.0,36 months,13.44,405.18,C,C3,Veterinary Tecnician,4 years,RENT,...,694.0,5.0,0.0,8822.0,68.4,6.0,w,Individual,0.0,0.0


## Feature engineering

### Remove feature with unique values

In [6]:
df.drop("id", axis=1, inplace=True)

# Remove emp_title to many different values

df = df.drop(labels="emp_title", axis=1)
df = df.drop("title", axis=1)
df = df.drop(labels=["zip_code", "addr_state"], axis=1)

    # Remove grade, redundant


In [7]:
# Remove redundant feature
df = df.drop("grade", axis=1)

### Convert feature

In [ ]:
# convert term to integer
df["term"] = df["term"].apply(lambda s: np.int8(s.split()[0]))

# Convert emp_length

df["emp_length"].replace(to_replace="10+ years", value="10 years", inplace=True)
df["emp_length"].replace("< 1 year", "0 years", inplace=True)


def emp_length_to_int(s):
    if pd.isnull(s):
        return s
    else:
        return np.int8(s.split()[0])


df["emp_length"] = df["emp_length"].apply(emp_length_to_int)

# Convert date features
df["earliest_cr_line"] = pd.to_datetime(df["earliest_cr_line"])
df["issue_d"] = pd.to_datetime(df["issue_d"])

# Convert subgrade to numeric
replacements = [
        ("A", "1"),
        ("B", "2"),
        ("C", "3"),
        ("D", "4"),
        ("E", "5"),
        ("F", "6"),
        ("G", "7"),
    ]
for r in replacements:
    df["sub_grade"] = df["sub_grade"].str.replace(r[0], r[1])
df["sub_grade"] = df["sub_grade"].astype(float)
df["sub_grade"] = df["sub_grade"].replace(
    np.sort(df["sub_grade"].unique()),
    np.arange(df["sub_grade"].unique().shape[0]).astype(int),
)

/tmp/ipykernel_165/381009928.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["emp_length"].replace(to_replace="10+ years", value="10 years", inplace=True)
/tmp/ipykernel_165/381009928.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method

In [ ]:
# THE TARGET
df["charged_off"] = (df["loan_status"] == "Charged Off").apply(np.uint8)
df = df.drop("loan_status", axis=1)


## Feature creation

In [ ]:
#  fico_range_low fico_range_high are correlated, take average

df["fico_score"] = 0.5 * df["fico_range_low"] + 0.5 * df["fico_range_high"]
df = df.drop(["fico_range_high", "fico_range_low"], axis=1)

# Feature creation


def diff_date_month(a, b):
    return 12 * (a.dt.year - b.dt.year) + (a.dt.month - b.dt.month)


def ratio_pub_rec_pub_rec_bankruptcies(pub_rec_bankruptcies, pub_rec):
    if pub_rec > 0:
        return pub_rec_bankruptcies / pub_rec
    else:
        return -1


df["month_of_year"] = df["issue_d"].dt.month - 1
df["ratio_loan_amnt_annual_inc"] = df["loan_amnt"] / df["annual_inc"]
df["ratio_open_acc_total_acc"] = df["open_acc"] / df["total_acc"]

df["month_since_earliest_cr_line"] = diff_date_month(
    df["issue_d"], df["earliest_cr_line"]
)
df = df.drop("earliest_cr_line", axis=1)

df["ratio_pub_rec_month_since_earliest_cr_line"] = (
    df["pub_rec"] / df["month_since_earliest_cr_line"]
)
df["ratio_pub_rec_bankruptcies_month_since_earliest_cr_line"] = (
    df["pub_rec_bankruptcies"] / df["month_since_earliest_cr_line"]
)
df["ratio_pub_rec_bankruptcies_pub_rec"] = df.apply(
    lambda x: ratio_pub_rec_pub_rec_bankruptcies(x.pub_rec_bankruptcies, x.pub_rec),
    axis=1,
)


### Missing values

In [ ]:
df = df.dropna()

### Feature type

In [ ]:
for e in [
    "open_acc",
    "total_acc",
    "emp_length",
    "mort_acc",
    "pub_rec_bankruptcies",
    "month_since_earliest_cr_line",
    "sub_grade",
]:
    df[e] = df[e].astype(int)

In [ ]:
# Categorical to numbers

for e in [
    "initial_list_status",
    "application_type",
    "home_ownership",
    "verification_status",
    "purpose",
]:
    unique_values = df[e].unique()
    df[e] = df[e].replace(unique_values, np.arange(len(unique_values)))

### Sort output by date

In [ ]:
df = df.sort_values(by="issue_d")

In [ ]:
df.head()

## Upload pandas dataframe to S3

In [ ]:
feature_filepath = f'{bucket_name}/{FEATURE_FILENAME}.csv'

df.to_csv(
    f's3://{feature_filepath}',
    storage_options={
        'client_kwargs': {'endpoint_url': MINIO_ENDPOINT}
    },
    index=False
)